# Artwork Label Generator

The generator creates fake titles and descriptions for non-existing work of art. Its built based on one of the [Implementations of early and well-known poetry generators](https://github.com/aparrish/rwet/blob/master/some-poetry-generators.ipynb) by Allison Parrish.

In [22]:
title_adjs = [
    "Cruel",
    "Diagonal",
    "Drowning",
    "Red",
    "Blue",
    "Dark",
    "Violet",
    "Crying",
    "Dotted",
    "Mellow"
]

In [40]:
keys = [
    "Girl",
    "Night",
    "Watch",
    "Snake",
    "Door",
    "Shadow",
    "River",
    "Despair",
    "Child",
    "Head",
    "Cloud",
    "American",
    "Greed",
    "Old man",
    "Machine",
    "Field",
    "Hill",
    "December",
    "Summer",
    "Railroad",
    "River",
    "Day"
]

In [24]:
first_medium = [
    "Oil",
    "Charcoal",
    "Acrylic",
    "Conté",
    "Monoprint",
    "Collage",
    "Engraving",
    "Metal",
    "Alginate",
    "Watercolor",
    "Ink",
    "Clay",
    "Soil",
    "Chalk",
    "Cloisonné",
    "Pencil"
]

In [25]:
second_medium = [
    "paper",
    "canvas",
    "wood",
    "ceramic"
]

In [29]:
advs = [
    'beautifully',
    'curiously',
    'eagerly',
    'fervently',
    'keenly',
    'passionately',
    'tenderly',
    'impressivley',
    'inensely',
    'emotionally',
    'likely'
]

In [30]:
verbs = [
    'contrasted',
    'gathered',
    'weaved together',
    'quilted',
    'collected',
    'united',
    'associated',
    'designed',
    'related',
    'created',
    'constructed',
    'composed',
    'intertwined',
    'incorporated',
    'introduced',
    'blended',
    'decorated',
    'stylized'
]

In [31]:
verbs2 = [
    'expresses',
    'presents',
    'speaks',
    'depicts',
    'characterizes',
    'illustrates',
    'interprets',
    'portrays',
    'represents',
    'narrates',
    'outlines'
]

In [16]:
import json
color_data = json.loads(open("xkcd.json").read())

def hex_to_int(s):
    s = s.lstrip("#")
    return int(s[:2], 16), int(s[2:4], 16), int(s[4:6], 16)
colors = dict()
for item in color_data['colors']:
    colors[item["color"]] = hex_to_int(item["hex"])

In [17]:
# testing
colors['pink']

(255, 129, 192)

In [19]:
import math
def distance(coord1, coord2):
    return math.sqrt(sum([(i - j)**2 for i, j in zip(coord1, coord2)]))
def subtractv(coord1, coord2):
    return [c1 - c2 for c1, c2 in zip(coord1, coord2)]
def addv(coord1, coord2):
    return [c1 + c2 for c1, c2 in zip(coord1, coord2)]
def meanv(coords):
    # assumes every item in coords has same length as item 0
    sumv = [0] * len(coords[0])
    for item in coords:
        for i in range(len(item)):
            sumv[i] += item[i]
    mean = [0] * len(sumv)
    for i in range(len(sumv)):
        mean[i] = float(sumv[i]) / len(coords)
    return mean

In [29]:
# testing
print(str(distance(colors['red'], colors['green'])) 
      + '\n' 
      + str(distance(colors['red'], colors['orange'])))

273.70787347096905
116.88028062936878


In [42]:
def closest(space, coord, n=5):
    closest = []
    for key in sorted(space.keys(),
                      key=lambda x: distance(coord, space[x]))[:n]:
        closest.append(key)
    return closest

In [106]:
# testing
import random
key_color = random.choice(list(colors.keys()))
new = closest(colors, colors[key_color])
new

['baby purple', 'pastel purple', 'lavender', 'lilac', 'liliac']

In [107]:
# testing2
print('he is', 
      key_color + '.', 
      'his', new[1], 'shirts and', new[2], 'smell,',
     'and', new[3])

he is baby purple. his pastel purple shirts and lavender smell, and lilac


In [8]:
import annoy
import spacy
import random
from itertools import islice
nlp = spacy.load('en_core_web_md')
qualified = [item for item in nlp.vocab if item.has_vector and item.is_alpha]

In [95]:
lexmap = []
t = annoy.AnnoyIndex(300)
for i, item in enumerate(islice(sorted(qualified, key=lambda x: x.prob, reverse=True), 100000)):
    t.add_item(i, item.vector)
    lexmap.append(item)
t.build(100)

True

In [96]:
def similarsemantic(t, nlp, word, n):
    seen = set()
    count = 0
    for i in t.get_nns_by_vector(nlp.vocab[word].vector, 100):
        this_word = lexmap[i].orth_.lower()
        if this_word not in seen and word != this_word:
            seen.add(this_word)
            count += 1
            yield this_word
            if count >= n:
                break

In [143]:
import textwrap
import random
year = random.randint(1993,2018)

key = random.choice(keys)
nouns = list(similarsemantic(t, nlp, key, 60))
nouns_len = len(nouns)
key2 = nouns[nouns_len-1]
nouns2 = list(similarsemantic(t, nlp, key2, 60))
# nouns2 = list(similarsemantic(t, nlp, 'art', 100))


output = random.choice(title_adjs) + " " + key + "\n"
output += random.choice(first_medium) + " " + "on" + " " + random.choice(second_medium) + ", " + str(year) + "\n"
output += "\n"


history = []
body = ""
for i in range(2):
    kind = random.choice(["short", "long"])
    if kind == "long":
        # adjectives and adverbs will be present only 50% of the time
        line = " ".join([
            "The",
            random.choice([random.choice(adjs), ""]),
            random.choice(nouns),
            random.choice([random.choice(advs), ""]),
            random.choice(verbs2), 
            " the",
            random.choice([random.choice(adjs), ""]),
            random.choice(nouns),
            random.choice(verbs),
            "with its",
            random.choice(nouns),
            "of",
            random.choice(nouns2)])
        body += line
    else:
        adj_noun = random.choice(adjs) + " " + random.choice(nouns)
        if len(history) > 0 and history[-1] == "short":
            body += "; " + adj_noun
        else:
            body += "The piece stands out as the " + adj_noun
    body += ". "
    history.append(kind)
# clean up output
body = body.replace("  ", " ")
body = body.replace(". ;", ";")
# put everything together
output += textwrap.fill(body, 50)

print(output)

Violet Hill
Pencil on paper, 2014

The piece stands out as the classic north. The
hills likely interprets the romantic northwest
designed with its southwest of crags.


In [144]:
import pycorpora

In [156]:
# test
crayola_data = pycorpora.colors.crayola
print(crayola_data["colors"][3]["color"])

Aquamarine


In [249]:
art_data = pycorpora.art.isms
adjs = art_data["isms"]
adjs

['abstract expressionism',
 'academic',
 'action painting',
 'aestheticism',
 'art deco',
 'art nouveau',
 'avant-garde',
 'barbizon school',
 'baroque',
 'bauhaus',
 'biedermeier',
 'caravaggisti',
 'carolingian',
 'classicism',
 'cloisonnism',
 'cobra',
 'color field painting',
 'conceptual art',
 'cubism',
 'cubo-futurism',
 'dada',
 'dadaism',
 'de stijl',
 'deformalism',
 'der blaue reiter',
 'die brücke',
 'divisionism',
 'eclecticism',
 'ego-futurism',
 'existentialism',
 'expressionism',
 'fauvism',
 'fluxus',
 'formalism',
 'futurism',
 'geometric abstraction',
 'gothic art',
 'gründerzeit',
 'hard-edge painting',
 'historicism',
 'hudson river school',
 'humanism',
 'hyperrealism',
 'idealism',
 'illusionism',
 'immagine&poesia',
 'impressionism',
 'incoherents',
 'installation art',
 'international gothic',
 'intervention art',
 'jugendstil',
 'kinetic art',
 'land art',
 'les nabis',
 'lettrism',
 'lowbrow',
 'luminism',
 'lyrical abstraction',
 'mail art',
 'manierism',
 '

In [172]:
year = random.randint(1993,2018)

key = random.choice(keys)
nouns = list(similarsemantic(t, nlp, key, 60))
nouns_len = len(nouns)
key2 = nouns[nouns_len-1]
nouns2 = list(similarsemantic(t, nlp, key2, 60))
# nouns2 = list(similarsemantic(t, nlp, 'art', 100))


output = random.choice(title_adjs) + " " + key + "\n"
output += random.choice(first_medium) + " " + "on" + " " + random.choice(second_medium) + ", " + str(year) + "\n"
output += "\n"


history = []
body = ""
for i in range(2):
    kind = random.choice(["short", "long"])
    if kind == "long":
        # adjectives and adverbs will be present only 50% of the time
        line = " ".join([
            "The",
            random.choice([random.choice(adjs),"<adj>", ""]),
            random.choice(nouns),
            random.choice([random.choice(advs), ""]),
            random.choice(verbs2), 
            " the",
            random.choice([random.choice(adjs),"<adj>", ""]),
            random.choice(nouns),
            random.choice(verbs),
            "with its",
            random.choice(nouns),
            "of",
            random.choice(nouns2)])
        body += line
    else:
        adj_noun = random.choice(adjs) + " " + random.choice(nouns)
        if len(history) > 0 and history[-1] == "short":
            body += "; " + adj_noun
        else:
            body += "The piece stands out as the " + adj_noun
    body += ". "
    history.append(kind)
# clean up output
body = body.replace("  ", " ")
body = body.replace(". ;", ";")
# put everything together
output += textwrap.fill(body, 50)

print(output)

Red Shadow
Acrylic on canvas, 2013

The piece stands out as the neo-impressionism
horizon. The caravaggisti billowing characterizes
the <adj> darkened created with its horizon of
applicator.


In [185]:
key = random.choice(keys)
nouns = list(similarsemantic(t, nlp, key, 60))
nouns2 = []
event_data = pycorpora.archetypes.event
for i in range(17):
    nouns2.append(event_data["events"][i]["name"])
nouns2

['arrival',
 'departure',
 'giving',
 'receiving',
 'discovery',
 'loss',
 'deceit',
 'victory',
 'defeat',
 'reversal',
 'enlightenment',
 'initiation',
 'expulsion',
 'ritual',
 'injury',
 'healing',
 'rebirth']

In [188]:
year = random.randint(1993,2018)
output = random.choice(title_adjs) + " " + key + "\n"
output += random.choice(first_medium) + " " + "on" + " " + random.choice(second_medium) + ", " + str(year) + "\n"
output += "\n"


history = []
body = ""
for i in range(2):
    kind = random.choice(["short", "long"])
    if kind == "long":
        # adjectives and adverbs will be present only 50% of the time
        line = " ".join([
            "The",
            random.choice([random.choice(adjs), ""]),
            random.choice(nouns),
            random.choice([random.choice(advs), ""]),
            random.choice(verbs2), 
            " the",
            random.choice([random.choice(adjs), ""]),
            random.choice(nouns),
            random.choice(verbs),
            "with its",
            random.choice(nouns),
            "of",
            random.choice(nouns2)])
        body += line
    else:
        adj_noun = random.choice(adjs) + " " + random.choice(nouns)
        if len(history) > 0 and history[-1] == "short":
            body += "; " + adj_noun
        else:
            body += "The piece stands out as the " + adj_noun
    body += ". "
    history.append(kind)
# clean up output
body = body.replace("  ", " ")
body = body.replace(". ;", ";")
# put everything together
output += textwrap.fill(body, 50)

print(output)

Dark Night
Conté on wood, 2014

The surrealism yesterday narrates the cobra next
collected with its yesterday of departure. The
conceptual art saturday characterizes the day
blended with its sunday of loss.


In [198]:
event_data["events"][3]["name"]
event_data["events"][4]["synonyms"][1]


'exposure'

In [200]:
key = random.choice(keys)
nouns = list(similarsemantic(t, nlp, key, 60))
nouns2 = []
nouns3 = []
event_data = pycorpora.archetypes.event
for i in range(17):
    nouns2.append(event_data["events"][i]["name"])
    nouns3.append(event_data["events"][i]["synonyms"])

In [201]:
nouns3

[['birth', 'introduction', 'reveal'],
 ['leaving', 'exiting', 'embarking', 'parting'],
 ['delivery', 'donation', 'presentation', 'endowment'],
 ['acceptance'],
 ['reveal', 'exposure'],
 ['separation', 'death', 'abandonment'],
 ['trickery', 'con', 'fool'],
 ['success', 'overcoming', 'triumph', 'passage'],
 ['fall', 'failure'],
 ['turnabout'],
 ['awakening', 'rise', 'improvement', 'learning', 'education'],
 ['acceptance', 'embracing', 'welcoming', 'joining'],
 ['separation', 'removal', 'rejection', 'elimination'],
 ['ceremony', 'rite', 'process'],
 ['wounding', 'hurting', 'insulting'],
 ['improvement', 'mending', 'joining'],
 ['renewal', 'return']]

In [238]:
key = random.choice(keys)
nouns = list(similarsemantic(t, nlp, key, 60))

events_random = random.randint(0, len(event_data["events"]))
nouns2 = event_data["events"][events_random]["name"]
nouns3 = event_data["events"][events_random]["synonyms"]
print(nouns2)
print(nouns3)

arrival
['birth', 'introduction', 'reveal']


In [257]:
art_data = pycorpora.art.isms
art = art_data["isms"]

In [264]:
year = random.randint(1993,2018)
output = random.choice(title_adjs) + " " + key + "\n"
output += random.choice(first_medium) + " " + "on" + " " + random.choice(second_medium) + ", " + str(year) + "\n"
output += "\n"


history = []
body = ""
for i in range(3):
    kind = random.choice(["short", "long"])
    if kind == "long":
        line = " ".join([
            "The",
            random.choice([random.choice(art), ""]),
            random.choice(nouns),
            random.choice([random.choice(art), ""]),
            random.choice(verbs2), 
            " the",
            nouns2,
            random.choice(verbs),
            "with its",
            random.choice(nouns3),
            "of",
            random.choice(nouns3)])
        body += line
    else:
        style = random.choice(art) + " piece"
        if len(history) > 0 and history[-1] == "short":
            body += "; " + style
        else:
            body += "The work stands out as the " + random.choice(art) + " style"
    body += ". "
    history.append(kind)
# clean up output
body = body.replace("  ", " ")
body = body.replace(". ;", ";")
# put everything together
output += textwrap.fill(body, 50)

print(output)

Diagonal Old man
Cloisonné on ceramic, 2012

The work stands out as the romanticism style;
neoclassicism piece. The classicism wrong
represents the arrival composed with its
introduction of reveal.
